This notebook demonstrates a multi-GRG phenotype simulation workflow, using the sim_phenotypes_multi_grg function to simulate both continuous and binary phenotypes across multiple GRG files representing 200 samples. The simulation works by sampling causal mutations from each GRG file (1000 per file), calculating genetic values for each individual based on a normal distribution model (mean=0, variance=1), then combining the genetic values from all GRGs additively and adding environmental noise based on a specified heritability parameter (0.33 in the examples). The notebook demonstrates several scenarios including loading all GRGs into RAM simultaneously versus sequentially and generating both continuous phenotypes and binary phenotypes with a specified population prevalence (0.2). We can also optionally saving the effect sizes to parameter files for downstream analysis.

In [1]:
from grg_pheno_sim.multi_grg_phenotype import sim_phenotypes_multi_grg
from grg_pheno_sim.model import grg_causal_mutation_model


The following command only serves the purpose of converting the VCF zip file into a GRG that will be used for the phenotype simulation. The bash script below will function as expected given the relative path for the source data file is accurate.

In [2]:
%%script bash --out /dev/null
if [ ! -f test-200-samples.grg ]; then
  grg construct -p 10 ../data/test-200-samples.vcf.gz --out-file test-200-samples.grg
fi


In [3]:
%%script bash --out /dev/null
if [ ! -f test-200-samples_copy.grg ]; then
  grg construct -p 10 ../data/test-200-samples.vcf.gz --out-file test-200-samples_copy.grg
fi

In [4]:
%%script bash --out /dev/null
if [ ! -f test-200-samples_last.grg ]; then
  grg construct -p 10 ../data/test-200-samples.vcf.gz --out-file test-200-samples_last.grg
fi

Processing input file in 10 parts.
Auto-calculating number of trees per part.
Converting segments of input data to graphs
100%|██████████| 10/10 [00:00<00:00, 21.62it/s]
Merging...


In [5]:
grg_list = ["test-200-samples.grg", "test-200-samples_copy.grg"]
#this is the list of GRG files to be loaded in 

We will first demonstrate loading all GRG files into RAM and simulating phenotypes. Causal mutations are sampled from each GRG, and the genetic values are obtained for the samples. The combined genetic dataframe is the addition of each GRG's genetic values. Noise is sampled at the end and added to obtain the phenotypes.

NOTE: It is necessary for each GRG to have the same number of samples.

In [6]:
model_type = "normal"
mean = 0
var = 1

model = grg_causal_mutation_model(model_type, mean=mean, var=var)

num_causal_per_file = 1000

random_seed = 1

normalize_phenotype = True #set check to True if we want phenotypes normalized

normalize_genetic_values_before_noise = False

heritability = 0.33

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

save_effects = True

path_list = ['first_sample_effect_sizes.par', 'second_sample_effect_sizes.par']

multi_grg_uni_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, normalize_phenotype=normalize_phenotype, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability, save_effect_output=save_effects, effect_path_list=path_list)


Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      18.989343                   0
1                1      -5.069896                   0
2                2      14.650330                   0
3                3      13.288474                   0
4                4       5.264167                   0
..             ...            ...                 ...
195            195       1.882760                   0
196            196      26.758005                   0
197            197       5.169511                   0
198            198      11.999558                   0
199            199      13.201295                   0

[200 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      -3.867656                   0
1                1      

In [7]:
multi_grg_uni_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,0.088907,-1.588574,-1.499667
1,0,1,-0.894698,-1.074980,-1.969678
2,0,2,0.700123,-0.197808,0.502314
3,0,3,-0.035625,-1.155047,-1.190673
4,0,4,-0.197840,0.096319,-0.101522
...,...,...,...,...,...
195,0,195,-0.433147,1.299625,0.866479
196,0,196,0.923034,0.813379,1.736413
197,0,197,-0.243216,-0.113907,-0.357123
198,0,198,0.618892,-0.973733,-0.354841


Now, we demonstrate a case where the function uses default values set for the parameters instead of custom values.

In [8]:
heritability = 0.33

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

multi_grg_uni_phenotypes_default = sim_phenotypes_multi_grg(grg_list, load_all_ram=load_all_into_RAM, heritability=heritability)


Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      37.653060                   0
1                1      11.098586                   0
2                2      -7.106104                   0
3                3      -4.611883                   0
4                4       9.838466                   0
..             ...            ...                 ...
195            195      17.875708                   0
196            196      11.002923                   0
197            197       9.613248                   0
198            198      13.016873                   0
199            199      12.360494                   0

[200 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0       1.591639                   0
1                1      

In [9]:
multi_grg_uni_phenotypes_default

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,39.244699,-10.230496,29.014203
1,0,1,2.838922,-31.588926,-28.750003
2,0,2,-12.454799,0.224910,-12.229888
3,0,3,10.457548,-9.853528,0.604020
4,0,4,10.635390,25.230372,35.865761
...,...,...,...,...,...
195,0,195,39.393934,7.503041,46.896975
196,0,196,29.190468,-45.296330,-16.105862
197,0,197,-0.530278,-39.620636,-40.150914
198,0,198,16.737333,-5.814789,10.922545


We now perform similar simulations, but by loading the GRGs into RAM sequentially (instead of all together).

In [10]:
model_type = "normal"
mean = 0
var = 1

model = grg_causal_mutation_model(model_type, mean=mean, var=var)

num_causal_per_file = 1000

random_seed = 1

normalize_phenotype = True #check to ensure phenotypes are normalized

normalize_genetic_values_before_noise = False

heritability = 0.33

load_all_into_RAM = False #this parameter decides whether to load all GRGs into RAM together

save_effects = True

path_list = ['first_seq_sample_effect_sizes.par', 'second_seq_sample_effect_sizes.par']

multi_grg_uni_seq_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, normalize_phenotype=normalize_phenotype, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability, save_effect_output=save_effects, effect_path_list=path_list)

Loaded test-200-samples.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      20.266713                   0
1                1       9.503610                   0
2                2      10.698492                   0
3                3      -0.769386                   0
4                4      -4.790418                   0
..             ...            ...                 ...
195            195       2.397429                   0
196            196       3.973493                   0
197            197     -11.270056                   0
198            198       2.464031                   0
199            199      20.250292                   0

[200 rows x 3 columns]
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0     -18.436698                   0
1                1     -

In [11]:
multi_grg_uni_seq_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,0.365456,0.604110,0.969566
1,0,1,0.053317,0.085162,0.138479
2,0,2,0.349158,0.339322,0.688480
3,0,3,-0.365735,-0.545003,-0.910739
4,0,4,-0.609459,-0.158019,-0.767477
...,...,...,...,...,...
195,0,195,0.023499,0.020034,0.043533
196,0,196,0.060654,0.520389,0.581043
197,0,197,-1.219011,-0.305610,-1.524621
198,0,198,0.510317,-0.164642,0.345674


Now, we demonstrate a case with binary phenotypes.

In [12]:
model_type = "normal"
mean = 0
var = 1

model = grg_causal_mutation_model(model_type, mean=mean, var=var)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

binary=True

population_prevalence = 0.2

heritability = 0.33

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

save_effects = True

path_list = ['first_seq_sample_effect_sizes.par', 'second_seq_sample_effect_sizes.par']

multi_grg_uni_bin_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, binary=binary, population_prevalence=population_prevalence, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability, save_effect_output=save_effects, effect_path_list=path_list)

Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      11.950391                   0
1                1      -5.893570                   0
2                2     -17.431317                   0
3                3      -1.001182                   0
4                4     -10.160646                   0
..             ...            ...                 ...
195            195       4.992049                   0
196            196      -2.448080                   0
197            197      15.685694                   0
198            198      13.947414                   0
199            199      -2.123071                   0

[200 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0       9.126079                   0
1                1      

In [13]:
multi_grg_uni_bin_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,21.076469,-45.968770,0
1,0,1,9.665183,-14.236230,0
2,0,2,17.757047,8.041464,1
3,0,3,-13.112307,-19.178292,0
4,0,4,4.446339,-3.876065,0
...,...,...,...,...,...
195,0,195,22.572750,9.931884,1
196,0,196,18.974627,-4.568096,1
197,0,197,33.082352,2.905909,1
198,0,198,36.626187,-28.479496,1


In [14]:
binary_list = multi_grg_uni_bin_phenotypes["phenotype"]
num_zeros = (binary_list == 0).sum()
num_ones = (binary_list == 1).sum()

print("Number of 0s:", num_zeros)
print("Number of 1s:", num_ones)
print("Population prevalence ratio observed: ", str(num_ones/(num_ones+num_zeros)))


Number of 0s: 52
Number of 1s: 148
Population prevalence ratio observed:  0.74


Finally, we demonstrate a case with more than 2 GRGs. 

In [15]:
new_grg_list = ["test-200-samples.grg", "test-200-samples_copy.grg", "test-200-samples_last.grg"]


In [16]:
model_type = "normal"
mean = 0
var = 1

model = grg_causal_mutation_model(model_type, mean=mean, var=var)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

heritability = 0.33

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

multi_grg_uni_phenotypes_three = sim_phenotypes_multi_grg(new_grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability)


Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Loaded test-200-samples_last.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      29.695807                   0
1                1      32.040705                   0
2                2      15.485066                   0
3                3      28.221312                   0
4                4      48.614576                   0
..             ...            ...                 ...
195            195      42.639202                   0
196            196      30.901051                   0
197            197      44.093588                   0
198            198      29.993675                   0
199            199      25.518254                   0

[200 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      -9.388209   

In [17]:
multi_grg_uni_phenotypes_three

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-19.664762,23.640757,3.975995
1,0,1,-17.382912,-1.198919,-18.581831
2,0,2,-36.235972,-5.999430,-42.235402
3,0,3,-12.555745,-35.555221,-48.110966
4,0,4,16.720715,15.865061,32.585776
...,...,...,...,...,...
195,0,195,-4.909940,6.128196,1.218257
196,0,196,-39.497186,46.087293,6.590107
197,0,197,-2.219644,70.047132,67.827488
198,0,198,-6.487518,33.698676,27.211158
